In [164]:
using PyPlot

In [165]:
# Convert an Float32 rbg image to Float64 grayscale image
function convert_to_grayscale(I::Array{Float32,3})
    I=convert(Array{Float64,3}, I)
    I_gray = 0.2989*I[:,:,1] + 0.5870*I[:,:,2] + 0.1140*I[:,:,3]
    return I_gray::Array{Float64,2}
end

convert_to_grayscale (generic function with 1 method)

In [166]:
# Load Tsukuba disparity dataset and convert it to grayscale
function load_data()
    i0_path = string(@__DIR__,"/i0.png")
    i0 = imread(i0_path)
    i0 = convert_to_grayscale(i0)
    i1_path = string(@__DIR__,"/i1.png")
    i1 = imread(i1_path)
    i1 = convert_to_grayscale(i1)
    gt_path = string(@__DIR__,"/gt.png")
    gt64 = convert(Array{Float64,2}, imread(gt_path)*255)

    @assert maximum(gt64) <= 16
    return i0::Array{Float64,2}, i1::Array{Float64,2}, gt64::Array{Float64,2}
end

load_data (generic function with 1 method)

In [167]:
function log_studentt(x::Float64, alpha::Float64, sigma::Float64)
    
    #TODO: Change the code to accept matrix input
    
    function log_student(x, alpha, sigma)
        return log((1 + (1/2*sigma^2)*x^2)^(-1*alpha))
    end
    
    function log_grad_student(x, alpha, sigma)
        return (-1*alpha*(1/sigma^2)*x) * (1 + (1/2*sigma^2)*x^2)^(-1*alpha - 2)
    end
    
    value = log_student(x, alpha, sigma)
    grad = log_grad_student(x, alpha, sigma)

    return value::Float64, grad::Float64
end

log_studentt (generic function with 1 method)

In [168]:
# Evaluate stereo log prior.
# Set: alpha=1.0, sigma=1.0
function stereo_log_prior(x::Array{Float64,2})
    alpha = 1.0
    sigma = 1.0
    value = 0
    grad = zeros(size(x))
    # Vertical potential
    for i = 1:size(x)[1]-1#Height?
        for j = 1:size(x)[2]#Width?
            value += log_studentt(x[i+1, j] - x[i,j], alpha, sigma)[1]
        end
    end 
    # Horizontal potential
    for i = 1:size(x)[1]#Height?
        for j = 1:size(x)[2]-1#Width?
            value += log_studentt(x[i, j+1] - x[i,j], alpha, sigma)[1]
        end
    end
    # Partial derivative to every pixel
    for k = 1:size(x)[1]
        for l = 1:size(x)[2]
            if k + 1 <= size(x)[1]
                grad[k,l] += log_studentt(x[k,l]-x[k+1,l], alpha, sigma)[2] 
            end
            if k - 1 >= 1
                grad[k,l] += log_studentt(x[k,l]-x[k-1,l], alpha, sigma)[2]
            end
            if l + 1 <= size(x)[2]
                grad[k,l] += log_studentt(x[k,l]-x[k,l+1], alpha, sigma)[2]
            end
            if l - 1 >= 1
                grad[k,l] += log_studentt(x[k,l]-x[k,l-1], alpha, sigma)[2]
            end
        end
    end
    return  value::Float64, grad::Array{Float64,2}
end


stereo_log_prior (generic function with 1 method)

In [169]:
# Evaluate stereo log likelihood.
# Set: Alpha = 1.0, Sigma = 0.004
function stereo_log_likelihood(x::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2})
    alpha = 1.0
    sigma = 0.004
    value = 0
    for i = 1:size(x)[1]
        for j = 1:size(x)[2]
            value += log_studentt((im0[i,j]-im1[i-trunc(Int, x[i,j]), j]), alpha, sigma)[1]
        end
    end
    #TODO: grad
    grad = zeros(size(x))
    return value::Float64, grad::Array{Float64,2}
end

stereo_log_likelihood (generic function with 1 method)

In [170]:
# Evaluate stereo posterior
function stereo_log_posterior(x::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2})




    return log_posterior::Float64, log_posterior_grad::Array{Float64,2}
end

stereo_log_posterior (generic function with 1 method)

In [171]:
# Run stereo algorithm using gradient ascent or sth similar
function stereo(x0::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2})




    return x::Array{Float64,2}
end

stereo (generic function with 1 method)

In [172]:
function problem3()
    # use problem 2's load_data
    im0, im1, gt = load_data()
    
    print(stereo_log_likelihood(gt, im0, im1)[1])
    # Display stereo: Initialized with constant 8's


    # Display stereo: Initialized with noise in [0,14]


    # Display stereo: Initialized with gt


    # Coarse to fine estimation..


end


problem3 (generic function with 1 method)

In [173]:
problem3()

-0.024335296835115088